In [1]:
%load_ext autoreload
%autoreload 2
import gym
import os
import numpy as np
import random
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from experiment import rollout, ReplayBuffer, Trajectory, load_model, save_model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
MODEL_NAME = 'model_v0_lunar_lander_v2'
HIDDEN = 64
curr_step = 0
writer = SummaryWriter()

class Behavior(torch.nn.Module):
    def __init__(self, input_shape, num_actions):
        super(Behavior, self).__init__()
        self.classifier = torch.nn.Sequential(
            nn.Linear(input_shape, HIDDEN), 
            nn.ReLU(),
            nn.Linear(HIDDEN, HIDDEN),      
            nn.ReLU(),            
            nn.Linear(HIDDEN, HIDDEN),      
            nn.ReLU(),            
            nn.Linear(HIDDEN, HIDDEN),       
            nn.ReLU(),
            nn.Linear(HIDDEN, num_actions)
        )        

    def forward(self, x):
        return self.classifier(x)

In [3]:
env = gym.make('LunarLander-v2')

In [4]:
loss_object = torch.nn.CrossEntropyLoss().to(device)
model_sample = Behavior(input_shape=env.observation_space.shape[0]+2, num_actions=env.action_space.n).to(device)
optimizer = torch.optim.Adam(model_sample.parameters())

In [5]:
rb = ReplayBuffer(max_size=500, last_few=200)

# Random rollout
trajectories, avg_reward, length = rollout(episodes=500, env=env, render=False)
rb.add(trajectories)


# Keep track of steps used during random rollout!
epoch, model_sample, optimizer, loss, steps = load_model(MODEL_NAME, model_sample, optimizer, device, train=True)
steps += length
save_model(MODEL_NAME, epoch, model_sample, optimizer, loss, steps)
writer.add_scalar('Mean_Reward', avg_reward, steps)


print(f"Average Episode Reward: {avg_reward}")

No checkpoint found. Creating new model.
Average Episode Reward: -188.05348802942174


In [6]:
batch_size = 1024

In [7]:
def train_step(model, inputs, targets):
    optimizer.zero_grad()    
    predictions = model(inputs)
    loss = loss_object(predictions, targets)
    
    loss.backward()
    optimizer.step()
    
    return loss

def action_fn(model, inputs, sample_action=True):
    action_logits = model(inputs)
    action_probs = torch.softmax(action_logits, axis=-1)

    if sample_action:        
        m = torch.distributions.categorical.Categorical(logits=action_logits)             
        action = int(m.sample().squeeze().cpu().numpy())        
    else:
        action = int(np.argmax(action_probs.detach().squeeze().numpy()))
    return action
    

In [10]:
# SAMPLE ACTIONS

loss_sum = 0
loss_count = 0

epochs = 1000000
epoch, model_sample, optimizer, loss, steps = load_model(MODEL_NAME, model_sample, optimizer, device, train=True)
print(steps)


for i in range(epoch, epochs+epoch):
    x, y = rb.sample(batch_size, device)    
    loss = train_step(model_sample, x, y)
    loss_sum += loss
    loss_count += 1
    
    writer.add_scalar('Loss/train', loss, i)
    
    if i % 500 == 0:        
        trajectories, mean_reward, length = rollout(100, env=env, model=model_sample, sample_action=True, replay_buffer=rb, 
                              device=device, action_fn=action_fn)
        rb.add(trajectories)
        
        steps += length
        avg_loss = loss_sum/loss_count
        save_model(MODEL_NAME, i, model_sample, optimizer, avg_loss, steps)        
        print(f"Average Episode Reward: {mean_reward}")        
        writer.add_scalar('Mean_Reward', mean_reward, steps)
        

    if i % 200 == 0:
        avg_loss = loss_sum/loss_count
        print(f'i: {i}, s: {steps}, Loss: {avg_loss}')
        
        save_model(MODEL_NAME, i, model_sample, optimizer, avg_loss, steps)

Existing model found. Loading from epoch 200, steps 55247 with loss: 1.3822327852249146
55247
i: 200, s: 55247, Loss: 1.3730357885360718
i: 400, s: 55247, Loss: 1.370059847831726
Average Episode Reward: -166.9602751013439
i: 600, s: 64823, Loss: 1.364400863647461
i: 800, s: 64823, Loss: 1.3592472076416016
Average Episode Reward: -127.95494410298126
i: 1000, s: 74555, Loss: 1.3553591966629028
i: 1200, s: 74555, Loss: 1.351264476776123
i: 1400, s: 74555, Loss: 1.3477166891098022
Average Episode Reward: -152.4982371611905
i: 1600, s: 83928, Loss: 1.3446227312088013
i: 1800, s: 83928, Loss: 1.3416590690612793
Average Episode Reward: -155.21560109716816
i: 2000, s: 93637, Loss: 1.3390635251998901
i: 2200, s: 93637, Loss: 1.336624026298523
i: 2400, s: 93637, Loss: 1.334437370300293
Average Episode Reward: -156.19432653350927
i: 2600, s: 103220, Loss: 1.3324995040893555
i: 2800, s: 103220, Loss: 1.330764651298523
Average Episode Reward: -140.99624651204715
i: 3000, s: 112630, Loss: 1.32910788

KeyboardInterrupt: 

In [ ]:
rb.sample_command()

In [ ]:
cmd = (280.085, 318.91295076177306)
rb.sample_command()
env = gym.make('LunarLander-v2')
e, model, _, l = load_model(name=MODEL_NAME, train=False, model=model_sample, optimizer=optimizer, device=device)

_, mean_reward = rollout(episodes=5, env=env, model=model_sample, sample_action=True, 
                      cmd=cmd, render=True, device=device, action_fn=action_fn)


print(f"Average Episode Reward: {mean_reward}")